# Legibility

## Libraries

In [1]:
from bs4 import BeautifulSoup
from nltk.stem import PorterStemmer
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
#Generate random integer values
from random import seed
from random import randint
import math
#Regular Expressions
import re 

In [2]:
A = np.array([[1, 2]])
B = np.array([[5, 6]])
A
B
np.concatenate((A, B), axis = 1)

array([[1, 2, 5, 6]])

## Functions

In [3]:
#Functions:
#The following function receives a string indicating the path to follow and
#and returns the data in the file.
def openFile(s):
    file = open(s)
    data_file = file.read()
    file.close()
    return data_file
#The following function receives the data from the stop word's file, splits them
#and extends them.
def cleanStop(sw):
    csw = [ts for ts in sw.split()]
    esw = ['.', ',',';',':', '/','"', '?', '!', '¡', '<', '>', 'El', '>El']
    csw.extend(esw)
    return csw
#   The following function utilizes beautiful soup to obtain the methodologies
#   and put them into
def bsMeth(s):
    # Use beautiful Soup to separate the methodologies
    with open(s) as fp:
        soup = BeautifulSoup(fp, 'xml')
    all = soup.find_all('Problema')

    counter = 1
    problem = ""
    for met in soup.find_all('Problema'):
        [s.extract() for s in met.findAll('Problema')]
        ######print("*Metodologia", counter)
        #####print(met)
        problem = problem + str(met) + "\n"
        #####print("\n")
        counter += 1
    return problem
#   The following function receives the methodologies and separates them. It
#   creates a dictionary-matrix with the following structure:
#   Methodologies-Number-Word
def sepMeth(justificacion):
    met_matrix = {}
    counter_met = 1
    for met in justificacion:
        met_vector = {}
        counter_word = 1
        met = [d for d in met.split()]
        for word in met:
            met_vector[counter_word] = word
            counter_word += 1
        if(met_vector):
            met_vector.popitem()
        met_matrix["Problema" + str(counter_met)] = met_vector
        counter_met += 1
    return met_matrix

#   The following function receives the Stop Words and the Methodologies and
#   removes the stop words from the methodologies (It basically returns the
#   methodologies clean.).
def cleanMeth(sw, meth):
    cleanMeth_matrix = {}
    for m, m_vector in meth.items():
        cleanMeth_vector = {}
        for num, word in m_vector.items():
            word = word.lower()
            word = re.sub('[^a-zñáéíóú]', '', word)
            if (word not in sw) and word:
                cleanMeth_vector[num] = word
        cleanMeth_matrix[m] = cleanMeth_vector
    return cleanMeth_matrix
#   The following function receives the clean methodologies and returns a
#   dictionary with the structure Methodologies-WordVector
def listMeth(meth):
    listMethMatrix = {}
    for m, m_vector in meth.items():
        listMethVector = []
        for num, word in m_vector.items():
            listMethVector.append(word)
        listMethMatrix[m] = listMethVector
    return listMethMatrix
#   The following function gets the frequency of each word and divides them by
#   the amount of words in the document.
def relFreq(meth):
    freqMethMatrix = {}
    for m, m_vector in meth.items():
        freqMethVector = {}
        sizeM = len(m_vector)
        freq = nltk.FreqDist(m_vector)
        for word, frequency in freq.most_common():
            if word not in freqMethVector:
#                 print('Frequency: {}, SizeM: {}'.format(frequency, sizeM))
                freqMethVector[word] = frequency/sizeM
        freqMethMatrix[m] = freqMethVector
    return freqMethMatrix
#   The following function opens the frequency for the most common words in
#   spanish and returns a dictionary with the following structure: Word-Freq
def mostCommon():
    f = open("frecuencia.txt", "r")
    mostCommonVector = {}
    for line in f:
        line = [d for d in line.split()]
        line[2] = line[2].replace(',', '')
        mostCommonVector[line[1]] = line[2]
    return mostCommonVector
#   The following function receives the dictionary with the sections and
#   returns a Dictionary-Matrix with the following form:
#   Section#-Word:LogFrequency
def logFreq(meth, common):
    logFreqMatrix = {}
    for m, methVector in meth.items():
        logFreqVector = {}
        for word in methVector:
            ####print("Word:", word)
            if (word in common):
                ####print("Word:", word)
                ####print("Value:", common[word])
                logFreqVector[word] = np.log(float(common[word]))
            else:
                logFreqVector[word] = 0.0
        logFreqMatrix[m] = logFreqVector
    return logFreqMatrix
#   The following function receives a single dictionary dictionary of specific
#   sections. It returns a vocabulary based on all the words that appear in the
#   the complete set of documents.
def obtainVocSingle(D):
    voc = []
    for s, svec in D.items():
        for word in svec:
            if word not in voc:
                voc.append(word)
    return voc
#   The following function receives two dictionaries of specific sections. It returns
#   It returns a vocabulary based on all the words that appear in the complete set of
#   documents.
def obtainVocDouble(d1, d2):
    voc = []
    for s, svec in d1.items():
        for word in svec:
            if word not in voc:
                voc.append(word)
    for s, svec in d2.items():
        for word in svec:
            if word not in voc:
                voc.append(word)
    return voc

#   The following function creates a vocabulary for all the words that appear in
#   justifications.
def obtainVoc(TSU, Lic, Maestria, Doctorado):
    vocabulary = []
    for m, mvec in TSU.items():
        for word in mvec:
            if word not in vocabulary:
                vocabulary.append(word)
    for m, mvec in Lic.items():
        for word in mvec:
            if word not in vocabulary:
                vocabulary.append(word)
    for m, mvec in Maestria.items():
        for word in mvec:
            if word not in vocabulary:
                vocabulary.append(word)
    for m, mvec in Doctorado.items():
        for word in mvec:
            if word not in vocabulary:
                vocabulary.append(word)
    return vocabulary
#   The following function creates a vocabulary for all the words that appear in
#   justifications.
def obtainVoc3(TSU, Lic, Maestria):
    vocabulary = []
    for m, mvec in TSU.items():
        for word in mvec:
            if word not in vocabulary:
                vocabulary.append(word)
    for m, mvec in Lic.items():
        for word in mvec:
            if word not in vocabulary:
                vocabulary.append(word)
    for m, mvec in Maestria.items():
        for word in mvec:
            if word not in vocabulary:
                vocabulary.append(word)
    return vocabulary
#   The following function utilizes the vocabulary to dimensionate the relative
#   frequency.
def dimRelFreq(matrix, voc):
    dimRelFreqMatrix = {}
    for j, jvec in matrix.items():
        dimRelFreqVec = {}
        for word in voc:
            if word in jvec:
                dimRelFreqVec[word] = jvec[word]
            else:
                dimRelFreqVec[word] = 0.0
        dimRelFreqMatrix[j] = dimRelFreqVec
    return dimRelFreqMatrix

#   The following function subcatenates two matrices, it subtracts one from the
#   vector from the other and return the result in the following form
#   Justification-ResultantVector
def substract(A, B):
    resultantMatrix = {}
    for (a, avec), (b, bvec) in zip(A.items(), B.items()):
        resultantVec = []
        for (wa, va), (wb, vb) in zip(avec.items(), bvec.items()):
            resultantVec.append(va-vb)
        resultantMatrix[a] = resultantVec
    return resultantMatrix
#   The following function substracts entrance by entrance two dictionary
#   vectors. It returns the result of the subtraction but in a vector, no longer
#   a dictionary.
def substractVec(A, B):
    resultantVec = []
    for (wa, va), (wb, vb) in zip(A.items(), B.items()):
        resultantVec.append(va-vb)
    return resultantVec
#   The following function receives two dictionary matrixes, and concatenates
#   their respective vectors row by row
def concatenate(A, B):
    resultantMatrix = []
    for(a, avec), (b, bvec) in zip(A.items(), B.items()):
        resultantVec = []
        resultantVec = avec + bvec
        ###print(len(resultantVec))
        resultantMatrix.append(resultantVec)
    return resultantMatrix
#   The following function receives two list-vectors and concatenates them in
#   the order A+B. It returns the concatenated vector.
def concatenateVec(A, B):
    resultantVec = A + B
    return resultantVec
#   The followign function receives two dictionary list-vector and concatenates
#   them vector-entry by vector-entry. It returns that concatenation.
def concatenateDictionaries(A, B):
    resultantMatrix = []
    for(a, avec), (b, bvec) in zip(A.items(), B.items()):
        resultantVec = []
        for (wa, va) in avec.items():
            resultantVec.append(va)
        for(wb, vb) in bvec.items():
            resultantVec.append(vb)
        resultantMatrix.append(resultantVec)
    return resultantMatrix
#   The following function receives a dictionary-dictionary and returns a
#   a dictionary-list.
def enlist(A):
    resultantMatrix = {}
    for (a, avec) in (A.items()):
        resultantVec = []
        for(w, v) in avec.items():
            resultantVec.append(v)
        resultantMatrix[a] = resultantVec
    return resultantMatrix
#   The following function receives a dictionary-vector and returns a
#   list.
def enlistVec(A):
    resultantVec = []
    for(w, v) in A.items():
        resultantVec.append(v)
    return resultantVec

#   The following function takes two matrixes A, B and returns a training matrix
#   and a classification matrix. The training matrix contains all the related
#   vectors in a single matrix.
def training(A, B):
    trainingMatrix = []
    classMatrix = []
    for av in A:
        trainingMatrix.append(av)
        classMatrix.append(-1.0)
    for bv in B:
        trainingMatrix.append(bv)
        classMatrix.append(1.0)
    return (trainingMatrix, classMatrix)
#   The following function takes fifty vectors from A, B matrixes and stores
#   them in a single matrix (This will be our training matrix). The Function
#   also creates a vector as the classification vector.
def trainingR(A, B, C, D, E, F):
    trainingMatrix = []
    classMatrix = []
    for av in A:
        trainingMatrix.append(av)
        classMatrix.append(-1.0)
    for cv in C:
        trainingMatrix.append(cv)
        classMatrix.append(-1.0)
    for ev in E:
        trainingMatrix.append(ev)
        classMatrix.append(-1.0)
    for bv in B:
        trainingMatrix.append(bv)
        classMatrix.append(1.0)
    for dv in D:
        trainingMatrix.append(dv)
        classMatrix.append(1.0)
    for fv in F:
        trainingMatrix.append(fv)
        classMatrix.append(1.0)
    return(trainingMatrix, classMatrix)
#   The following function receives two dictionaries and returns the vectors
#   in the dictionary concatenated with the other matrix and a classification
#   matrix.
def trainingL(A, B):
    tMatrix = []
    cMatrix = []
    for a, avec in A.items():
        tMatrix.append(avec)
        cMatrix.append(-1.0)
    for b, bvec in B.items():
        tMatrix.append(bvec)
        cMatrix.append(1.0)
    return (tMatrix, cMatrix)
def trainingLR(A, B, C, D, E, F):
    trainingMatrix = []
    classMatrix = []
    for a, avec in A.items():
        trainingMatrix.append(avec)
        classMatrix.append(-1.0)
    for c, cvec in C.items():
        trainingMatrix.append(cvec)
        classMatrix.append(-1.0)
    for e, evec in E.items():
        trainingMatrix.append(evec)
        classMatrix.append(-1.0)
    for b, bvec in B.items():
        trainingMatrix.append(bvec)
        classMatrix.append(1.0)
    for d, dvec in D.items():
        trainingMatrix.append(dvec)
        classMatrix.append(1.0)
    for f, fvec in F.items():
        trainingMatrix.append(fvec)
        classMatrix.append(1.0)
    return(trainingMatrix, classMatrix)
###############################################
#           Science Contribution              #
###############################################

#   The following function receives a dimensionalized vector from one of the
#   possible classes of justifications and returns a test vector.
def obtainTest(A, B):
    # seed random number generator
    seed(1)
    # generate the random number
    r = randint(0, len(A))
    counter = 0
    testRF = {}
    testLF = {}
    for (wordA, vwordA), (wordB, vwordB) in zip(A.items(), B.items()):
        if counter == r:
            testRF = vwordA
            testLF = vwordB
            impWord = wordA
        counter += 1
    A.pop(impWord)
    B.pop(impWord)
    return (testRF, testLF)

#   The following function receives a Matrix and returns a random vector in it.
#   This vector will be use as a representative vector of the class in order
#   to classify a text file as easier or more difficult than it.
def random(A, B):
    seed(1)
    r = randint(0, len(A))
    counter = 0
    randRF = []
    randLF = []
    for (wordA, vwordA), (wordB, vwordB) in zip(A.items(), B.items()):
        if counter == r:
            randRF = vwordA
            randLF = vwordB
            impWord = wordA
        counter += 1
    A.pop(impWord)
    B.pop(impWord)
    return(randRF, randLF)
#   The following function receives a set of vectors and returns a random vector
#   from the collection.
def randomR(A):
    seed()
    print("Length of vector for random choice:", len(A))
    random = randint(0, len(A)-1)
    randVector = A[random]
    return randVector, random
#   The following function receives a set of matrixes and returns a list of random
#   vectors, one for each matrix.
def obtainRandomVectorsR(A, B, C, D):
    vecList = []
    vecPosition = []
    randomVecA, randomPositionA = randomR(A)
    vecList.append(randomVecA)
    vecPosition.append(randomPositionA)
    randomVecB, randomPositionB = randomR(B)
    vecList.append(randomVecB)
    vecPosition.append(randomPositionB)
    randomVecC, randomPositionC = randomR(C)
    vecList.append(randomVecC)
    vecPosition.append(randomPositionC)
    return vecList
#   The following function receives a set of matrixes and returns a list of random
#   vectors, one for each matrix.
def obtainRandomVectors3(A, B, C):
    vecList = []
    vecPosition = []
    randomVecA, randomPositionA = randomR(A)
    vecList.append(randomVecA)
    vecPosition.append(randomPositionA)
    randomVecB, randomPositionB = randomR(B)
    vecList.append(randomVecB)
    vecPosition.append(randomPositionB)
    randomVecC, randomPositionC = randomR(C)
    vecList.append(randomVecC)
    vecPosition.append(randomPositionC)
    return vecList, vecPosition
#   The following function receives eight matrixes and returns two lists: the
#   first list contains all the random RF vectors and the second list contains
#   LF vectors.
def obtainRandomVectors(A, B, C, D, E, F, G, H):
    vecListRF = [None]*4
    vecListLF = [None]*4
    (vecListRF[0], vecListLF[0]) = random(A, B)
    (vecListRF[1], vecListLF[1]) = random(C, D)
    (vecListRF[2], vecListLF[2]) = random(E, F)
    (vecListRF[3], vecListLF[3]) = random(G, H)

    return (vecListRF, vecListLF)
#   The following function receives eight matrixes two test vector related to
#   RF and LF, the general training matrix and its classyfying vector. The
#   function returns the maximum level of difficulty of the text.
def obtainGrade(lRF, lLF, testRF, testLF, tMatrix, CV):
    #Train the SVM
    clf = svm.SVC()
    clf.fit(tMatrix, CV)

    grade = 0
    goodGrade = 0
    ##print(testRF)
    for (vecRF, vecLF) in zip(lRF, lLF):
        testSubRF = substractVec(testRF, vecRF)
        testSubLF = substractVec(testLF, vecLF)
        conTest = concatenateVec(testSubRF, testSubLF)
        result = clf.predict([conTest])
        #print("Resultado: ")
        #print(result[0])
        if (result[0] == -1.0 or goodGrade>=3):
            #print("Entering results:")
            #print("Grade: ", grade)
            if(grade == 0):
                print("Your text is as good as TSU.")
            elif(grade == 1):
                print("Your text is as good as Lic.")
            elif(grade == 2):
                print("Your text is as good as Maestria.")
            elif(grade >= 3):
                print("Your text is as good as Doctorado.")
            break
        else:
            goodGrade += 1

        grade = grade + 1
#   The following function receives a list with the respective centroids, a
#   vector test in order to try and the training matrix with its respective
#   classes.
def obtainGradeR(centroidList, testRF, testLF, tMatrix, CV):
    clf = svm.SVC()
    clf.fit(tMatrix, CV)

    grade = 0
    goodGrade = 0
    #print(testRF)
    #print(len(testRF))
    testRF = enlistVec(testRF)
    testLF = enlistVec(testLF)
    conTest = concatenateVec(testRF, testLF)
    conTest = np.array(conTest)
    #print(conTest)
    #print(len(conTest))
    result = clf.predict([conTest])
    for vec in centroidList:
        vec = np.array(vec)
        #print(vec)
        #print(len(vec))
        subVec = conTest - vec
        #print(subVec)
        #print(len(subVec))
        result = clf.predict([subVec])
        #print(result)
        if (result[0] == -1.0 or goodGrade>=3):
            if(grade == 0):
                print("Your text is as good as TSU.")
            elif(grade == 1):
                print("Your text is as good as Lic.")
            elif(grade == 2):
                print("Your text is as good as Maestria.")
            elif(grade >= 3):
                print("Your text is as good as Doctorado.")
            break
        else:
            goodGrade += 1

        grade = grade + 1
#   The following function receives a test vector and a list of random vectors.
#   It returns the grade of the vector.
def obtainGradeRandom(vector, vector_list, clf):
    grade = 0
    goodGrade = 0
    for vec in vector_list:
        vector = vector - vec
        result = clf.predict([vector])
        #print("Result: ", result)
        if (result[0] == -1.0 or goodGrade>=3):
            if(grade == 0):
                #print("Your text is as good as TSU.")
                return 0
            elif(grade == 1):
                #print("Your text is as good as Lic.")
                return 1
            elif(grade >= 2):
                #print("Your text is as good as Maestria.")
                return 2
            break
        else:
            goodGrade += 1

        grade = grade + 1
#   The following function receives a testMatrix, its vector classification and a
#   list of random vector. It returns the accuracy of the evaluator.
def randomEvaluator(test_matrix, classification, vector_list, clf):
    acc = 0
    print("Test Matrix: ")
    print(test_matrix)
    print("Length of Random: ", len(test_matrix))
    for i in range(len(test_matrix)):
        grade = obtainGradeRandom(test_matrix[i], vector_list, clf)
        #print("Grade: ", grade)
        #print("Classification: ", int(classification[i]))
        if grade == int(classification[i]):
            acc += 1
    return acc/len(test_matrix)
# #   The following function receives a testMatrix, its vector classification and a
# #   list of random vector and the plain justifications. It returns the justification
# #   and its respective classification wether it was wrong or not.
def getJustificationEvaluation(test_matrix, classification, vector_list, clf, justificacionTSU, justificacionLic, justificacionMaestria):
    acc = 0
    print("Test Matrix: ")
    print(test_matrix)
    print("Length of Random: ", len(test_matrix))
    counter = 1
    for i in range(len(test_matrix)):
        grade = obtainGradeRandom(test_matrix[i], vector_list, clf)
        #print("Grade: ", grade)
        #print("Classification: ", int(classification[i]))
        if counter <= 11:
            print("Justificacion de TSU:")
            if grade == int(classification[i]):
                print("Justificacion Correcta: ")
                print(justificacionTSU[counter])
                acc += 1
            else:
                print("Justificacion Incorrecta: ")
                print(justificacionTSU[counter])
            counter += 1
        elif 12<= counter <=23:
            print("Justificaciones de Licenciatura")
            if grade == int(classification[i]):
                print("Justificacion Correcta: ")
                print(justificacionLic[counter])
                acc += 1
            else:
                print("Justificacion Incorrecta: ")
                print(justificacionLic[counter])
            counter += 1
        else:
            print("Justificaciones de Maestria: ")
            if grade == int(classification[i]):
                print("Justificacion Correcta: ")
                print(justificacionMaestria[counter])
                acc += 1
            else:
                print("Justificacion Incorrecta: ")
                print(justificacionMaestria[counter])
            counter += 1

    return acc/len(test_matrix)
#   The following function receives all the justifications for all the different
#   scholar levels and a list for the positions of the random vectors.
#   It returns the random selected vectors and prints them in screen.
def getRandomJustifications(list, A, B, C):
    print("TSU random justification:")
    print(A[list[0]+6])
    print("Lic. random justification: ")
    print(A[list[1]+6])
    print("Maestria random justification: ")
    print(A[list[2]+6])


#   The following function receives two matrixes, one related to the Relative
#   Frequency vectors and one related to the Logarithmic Frequency. It concatenates
#   them, tranforms them into a vector instead of a dictionary and returns the
#   centroid of them all.
def centroid(A, B):
    A = enlist(A)
    B = enlist(B)
    M = concatenate(A, B)
    #Transform into a numpy array
    Marray = np.array(M)
    length = len(Marray)
    centroid = np.zeros(33642)
    for vec in Marray:
        centroid = np.add(centroid, vec)
    centroid = centroid*(1/len(M[0]))
    return centroid

def centroidR(A, B):
    A = enlist(A)
    B = enlist(B)
    M = concatenate(A, B)
    length = len(M)
    centroid = np.zeros(33642)
    for i in range(length):
        for j in range(len(M[0])):
            centroid[j] = centroid[j] + M[i][j]
    for j in range(len(M[0])):
        centroid[j] = centroid[j]/(len(M[0]))
    return centroid
#   The following function receives a matrix and returns its centroid.
def centroidRR(A):
    centroid = np.zeros(len(A[0]))
    for vec in A:
        centroid = centroid + vec
    return centroid/(len(A))
#   The following function receives the test matrix, the classification vector,
#   the centroid list and the svm classifier. It returns the accuracy of the centroid
#   evaluator.
def centroidEvaluator(test_matrix, classification, centroids, clf):
    acc = 0
    for i in range(len(test_matrix)):
        grade = obtainGradeRandom(test_matrix[i], centroids, clf)
        #print("Grade: ", grade)
        #print("Classification: ", int(classification[i]))
        if grade == int(classification[i]):
            acc += 1
    return acc/len(test_matrix)
#   The following function receives four matrixes and returns a list of centroids.
#   One for each matrix.
def obtainCentroids(A, B, C, D):
    list = []
    list.append(centroidRR(A))
    list.append(centroidRR(B))
    list.append(centroidRR(C))
    list.append(centroidRR(D))
    return list
#   The following function receives four matrixes and returns a list of centroids.
#   One for each matrix.
def obtainCentroids3(A, B, C):
    list = []
    list.append(centroidRR(A))
    list.append(centroidRR(B))
    list.append(centroidRR(C))
    return list
#   The following function removes the len zero vectors from the dictionary and
#   returns a dictionary.
def removeZero(GM):
    l = []
    for n, w_vec in GM.items():
        if len(w_vec) == 0:
            l.append(n)
    for e in l:
        del GM[e]
    return GM

#   The follwing function receives a matrix and eliminates the zero lenght vectors
#   from it.
def cleanVector(A):
    return removeZero(A)

#   The following function receives the four grade matrixes and eliminates the
#   zero length vectors from it.
def cleanVectors(A, B, C, D):
    
    return removeZero(A), removeZero(B), removeZero(C), removeZero(D)
#   The following function gets the size of the smallest vector in the dictionary
#   vector-word matrix.
def smallest(GM):
    smallestValue = 1000000
    for n, w_vec in GM.items():
        if smallestValue > len(w_vec):
            smallestValue = len(w_vec)
    return smallestValue
#   The following function gets the size of the biggest vector in the dictionry
#   vector-word matrix.
def biggest(GM):
    biggestValue = 0
    for n, w_vec in GM.items():
        if biggestValue < len(w_vec):
            biggestValue = len(w_vec)
    return biggestValue
#   The following function receives a grade matrix and returns the average size
#   of the vectors in it.
def average(GM):
    averageValue = 0
    allElements = len(GM)
    for n, w_vec in GM.items():
        averageValue = averageValue + len(w_vec)
    return (averageValue/allElements)
#   The following function receives a grade matrix and returns a dictionary with
#   the size of the smallest vector, the biggest vector and the average sizes
#   of the vectors.
def returnSizes(GM):
    sizes = {}
    sizes['Smallest'] = smallest(GM)
    sizes['Biggest'] = biggest(GM)
    sizes['Average'] = average(GM)
    return sizes
#   The following function receives two grade matrixes dimensionalized and concatenated
#   and returns the substraction of both of them.
def subInc(A, B):
    matrix = []
    A = np.array(A)
    B = np.array(B)
    for vecA in A:
        for vecB in B:
            matrix.append(vecA - vecB)
    return matrix
#   The following function receives two matrices dimensionalized and concatenated
#   and returns their conmutative combination in two differente matrices. The plusOne
#   and the minusOne.
def comData(A, B):
    plusOne = subInc(B, A)
    minusOne = subInc(A, B)
    return plusOne, minusOne

#   The following function receives two matrixes dimensionalized and concatenated with
#   vectors with the form: local-global. It returns a matrix which concatenates each vector
#   of the first matrix with eache vector of the second one, thus returning a matrix with 
#   vectors with form: localA-globalA-localB-globalB.
def conMatrixes(A, B):
    concatenated_matrix = []
    A = np.array(A)
    B = np.array(B)
    for vecA in A:
        for vecB in B:
            concatenated_matrix.append(np.hstack((vecA, vecB)))
    return concatenated_matrix
    
#   The following function receives two matrices dimensionalized and concatenated
#   with the following form of vectors: (Local and Global). It concatenates the matrixes
#   in both orders PlusOne and MinusOne and returns matrixes with vectors with the 
#   following form: localA-globalA-localB-globalB and localB-globalB-localA-globalA.
def comConData(A, B):
    plusOneMatrix = conMatrixes(B, A)
    minusOneMatrix = conMatrixes(A, B)
    return plusOneMatrix, minusOneMatrix

#   The following function receives a matrix and returns the 80 percent of the values
#   in one matrix and the other twenty in another matrix.
def eightyTwenty(A):
    m = np.array([])
    twenty = math.ceil(len(A)*(1/5))
    for i in range(twenty):
        np.append(m, A[i], 0)
        np.delete(A, i, 0)
    return A, m
#   The following function receives the plusOne and minusOne version for all the grades
#   and returns the training matrix along with its classification vector.
def allTogetherNow(POT, MOT, POL, MOL, POM, MOM):
    M = np.array([])
    y = np.array([])
    M = np.append(POT, POL, 0)
    M = np.append(M, POM, 0)
    M = np.append(M, MOT, 0)
    M = np.append(M, MOL, 0)
    M = np.append(M, MOM, 0)
    y1 = np.ones(len(POT) + len(POL) + len(POM))
    y2 = np.zeros(len(MOT) + len(MOL) + len(MOM))
    y2 = y2-1
    y = np.append(y1, y2, 0)
    return M, y
#   The following function receives plusOne and minusOne version for all the grades
#   and returns the training matrix along with its classification vector.
def allTogetherNow(POT, MOT, POL, MOL):
    M = np.array([])
    y = np.array([])
    M = np.append(POT, POL, 0)
    M = np.append(M, MOT, 0)
    M = np.append(M, MOL, 0)
    y1 = np.ones(len(POT) + len(POL))
    y2 = np.zeros(len(MOT) + len(MOL))
    y2 = y2-1
    y = np.append(y1, y2, 0)
    return M, y
#   The following function receives a plusOne and a minusOne matrix. It returns a 
#   an appended matrix of the both along with its classification vector.
def allTogetherNowDouble(PO, MO):
    M = np.array([])
    y = np.array([])
    M = np.append(PO, MO, axis = 0)
    y1 = np.ones(len(PO))
    y2 = np.zeros(len(MO))
    y = np.append(y1, y2, axis = 0)
    return M, y

#   The following function receives a dictionary-list matrix and returns eighty
#   percent of the vectors in one matrix and the other twenty percent in another
#   matrix.
def divideEightyTwenty(M):
    twenty = {}
    eighty = {}
    counter = 0
    stop = int(len(M)/5)
    for v, vvec in M.items():
        if counter <= stop:
            twenty[v] = vvec
        else:
            eighty[v] = vvec
        counter += 1
    return twenty, eighty
#   The following function receives two matrixes the plusOne and the minusOne and
#   returns it's respective trainingMatrix with their related classification vector.
def togetherNow(A, B):
    M = np.array([])
    y = np.array([])
    M = np.append(A,B,0)
    y1 = np.ones(len(A))
    y2 = np.zeros(len(B))
    y2 = y2-1
    y = np.append(y1, y2, 0)
    return M, y
#   The following function divides our whole training set into 80 percent for
#   training and 20 percent for testing. It returns both matrixes.
def getEightyTwenty(M, y):
    print("Enter Eighty Twenty")
    np.c_[M, y]
    length = len(M)
    testMatrix = np.array([])
    print(int(length*(1/5)))
    seed(1)
    for i in range(int(length*(1/5))):
        print(i)
        random = randint(0, length)
        np.append(testMatrix, M[random])
        np.delete(M, random, 0)
    with open('traingMatrix.txt', 'wb') as f:
        for line in trainingMatrix:
            np.savetxt(f, line, fmt = '%.2f')
    return trainingMatrix, testMatrix
#   The following function receives the M matrix which corresponds to the training
#   objects and the test matrix. It first trains the SVM for later test the accuracy
#   of it five times. Finally it returns the five accuracies along with its standard
#   deviation.
def testAccuracy(M, tM):
    #Get the training Matrix just values
    M = np.array(M)
    y = M[:, len(M[0])-1]
    M = np.delete(M, len(M[0])-1, 1)
    yt = tM[:, len(tM[0])-1]
    tM = np.delete(tM, len(tM[0])-1, 1)
    # Train the svm
    clf = svm.SVC()
    clf.fit(M)
    for i in range(5):
        counter = 0
        for j in range(len(tM)):
            if clf.predict(tM[i]) == y[i]:
                counter += 1
        accuracy.append(counter/len(tM))
    return accuracy
#   The following function receives four matrixes and constructs a single matrix
#   with the vectors of all the other matrixes and a vector with their grade
#   classification.
def testEvaluatorMatrix(A, B, C, D):
    M = np.array([])
    M = np.concatenate((A, B), axis = 0)
    M = np.concatenate((M, C), axis = 0)
    M = np.concatenate((M, D), axis = 0)
    y = np.array([])
    y1 = np.zeros(len(A))
    y2 = np.zeros(len(B)) + 1
    y3 = np.zeros(len(C))  + 2
    y4 = np.zeros(len(D)) + 3
    y = np.concatenate((y1, y2), axis = 0)
    y = np.concatenate((y, y3), axis = 0)
    y = np.concatenate((y, y4), axis = 0)
    return M, y
#   The following function receives four matrixes and constructs a single matrix
#   with the vectors of all the other matrixes and a vector with their grade
#   classification.
def testEvaluatorMatrix3(A, B, C):
    M = np.array([])
    M = np.concatenate((A, B), axis = 0)
    M = np.concatenate((M, C), axis = 0)
    y = np.array([])
    y1 = np.zeros(len(A))
    y2 = np.zeros(len(B)) + 1
    y3 = np.zeros(len(C))  + 2
    y = np.concatenate((y1, y2), axis = 0)
    y = np.concatenate((y, y3), axis = 0)
    return M, y
#   The following function receives a number (amount of specific sections to retrieve) and
#   a dictionary of specific sections.
def getNVector(n, A):
    rA = {}
    counter = 0
    stop = n
    for (a, avec) in A.items():
        if counter < stop: 
            rA[a] = avec
        else:
            break
        counter += 1
    return rA
#   The following function receives four matrixes and a number. It returns the
#   amount of vectors in the number for each of the matrixes.
def getNVectors(n, A, B, C, D):
    rA = {}
    rB = {}
    rC = {}
    rD = {}
    for (a, avec), (b, bvec), (c, cvec), (d, dvec) in zip (A.items(), B.items(), C.items(), D.items()):
        rA[a] = avec
        rB[b] = bvec
        rC[c] = cvec
        rD[d] = dvec
    return rA, rB, rC, rD
#   The following function receives three matrixes and a number. It returns the
#   amount of vectors in the number for each of the matrixes.
def get3NVectors(n, A, B, C):
    rA = {}
    rB = {}
    rC = {}
    counter = 0
    stop = n
    for (a, avec), (b, bvec), (c, cvec) in zip (A.items(), B.items(), C.items()):
        if counter < stop:
            rA[a] = avec
            rB[b] = bvec
            rC[c] = cvec
        else:
            break
        counter+=1
    return rA, rB, rC
#   The following function receives two vectors, actual results for the test vector and the
#   predicted vector produced by our classifier, it then returns the accuracy report for
#   our classifier. Accuracy considered as number of correct predictions divided by total
#   number of predictions made.
def obtain_accuracy(a, b):
    correct = np.where(a == b)
    return np.size(correct)/np.size(a)
    

#   Open the stop words file.
sw = openFile("stopWords.txt")
#   Curate the stop words.
sw = cleanStop(sw)
sw

['a',
 'al',
 'algo',
 'algunas',
 'algunos',
 'ante',
 'antes',
 'como',
 'con',
 'contra',
 'cual',
 'cuando',
 'de',
 'del',
 'desde',
 'donde',
 'durante',
 'e',
 'el',
 'ella',
 'ellas',
 'ellos',
 'en',
 'entre',
 'era',
 'erais',
 'eran',
 'eras',
 'eres',
 'es',
 'esa',
 'esas',
 'ese',
 'eso',
 'esos',
 'esta',
 'estaba',
 'estabais',
 'estaban',
 'estabas',
 'estad',
 'estada',
 'estadas',
 'estado',
 'estados',
 'estamos',
 'estando',
 'estar',
 'estaremos',
 'estará',
 'estarán',
 'estarás',
 'estaré',
 'estaréis',
 'estaría',
 'estaríais',
 'estaríamos',
 'estarían',
 'estarías',
 'estas',
 'este',
 'estemos',
 'esto',
 'estos',
 'estoy',
 'estuve',
 'estuviera',
 'estuvierais',
 'estuvieran',
 'estuvieras',
 'estuvieron',
 'estuviese',
 'estuvieseis',
 'estuviesen',
 'estuvieses',
 'estuvimos',
 'estuviste',
 'estuvisteis',
 'estuviéramos',
 'estuviésemos',
 'estuvo',
 'está',
 'estábamos',
 'estáis',
 'están',
 'estás',
 'esté',
 'estéis',
 'estén',
 'estés',
 'fue',
 'f

In [4]:
!ls

AvancesRecuperacionDeInformacion-3.pdf
Doctorado.xml
DoctoradoCompleto.xml
LegibilityRevisedJustification.py
Licenciatura.xml
LicenciaturaCompleto.xml
Maestria.xml
MaestriaCompleto.xml
ResultsTablesForClassificationI.ipynb
SoftwareRevisionJustification.ipynb
SoftwareRevisionProblem.ipynb
SoftwareRevisionProblemConcatenation.ipynb
SoftwareRevisionResults.ipynb
TSU.xml
TSUCompleta.xml
Untitled.ipynb
Untitled1.ipynb
completeTrainingData.txt
frecuencia.txt
justificacionDoctorado.xml
justificacionLic.xml
justificacionMaestria.xml
justificacionTSU.xml
obatinMet9.0.py
obtainMet.py
obtainMet1.0.py
obtainMet10.0.py
obtainMet2.0.py
obtainMet3.0.py
obtainMet4.0.py
obtainMet5.0.py
obtainMet6.0.py
obtainMet6.0py.py
obtainMet7.0.py
obtainMet8.0.py
obtainMet9.0.py
rp_svm.py
stopWords.txt
tesis.xml


# Main

In [5]:
s = 'TSUCompleta.xml'
with open(s) as fp:
    soup = BeautifulSoup(fp, 'xml')
    all = soup.find_all('Problema')
soup.find_all('Problema')

counter = 1
problema = ""
for met in soup.find_all('Problema'):
    [s.extract() for s in met.findAll('Problema')]
    ######print("*Metodologia", counter)
    #####print(met)
    problema = problema + str(met) + "\n"
    #####print("\n")
    counter += 1
problema

'<Problema>AUTOMATIZACIÓN DEL MÓDULO\nDE CAJA.El proceso que se realiza en la actualidad no permite conocer la información\noportunamente, ya que el tiempo que se llevan en recopilar la información depende\nde las personas involucradas en el proceso, además de que se presta a errores y\nmalos manejos.\nEl usuario tiene que trasladarse hasta las oficinas administrativas de la empresa para\nllenar la solicitud, y para llevar el control de la comprobación de los gastos lo hacen a\ntravés de una hoja de Excel, de la cual si no actualizan sus formulas, generan datos\nincorrectos.\nEn cuestión de almacenamiento lo único que se guarda directo al servidor es el\nmonto de dinero solicitado durante el día, pero no la información generada de cada\nuno de los solicitantes que es importante para el seguimiento del proceso.\nPara el departamento de contabilidad es bastante laborioso, ubicar cada uno de los\ndiferentes gastos en las cuentas contables al final del mes cuando generan la póliza,\ndebido

## Obtain Problem

In [6]:
problemTSU = bsMeth('TSUCompleta.xml')
problemLic = bsMeth('LicenciaturaCompleto.xml')
problemMasters = bsMeth('MaestriaCompleto.xml')
problemPhd = bsMeth('DoctoradoCompleto.xml')
problemTSU
problemLic
problemMasters
problemPhd

'<Problema>La gran cantidad de información disponible en Internet, la complejidad y heterogeneidad de la\nmisma, junto con las distintas características de los usuarios que acceden a dicha información, así\ncomo la gran competencia existente en la red, hacen crítico que el desarrollo de aplicaciones\nbasadas en Web, y de manera particular los sitios Web educativos deban contemplar no solo\naspectos de funcionalidad sino también de usabilidad, tomando en cuenta que estas aplicaciones no\nsolo deben tener como objetivo poner a disposición de los usuarios información exacta, sino orientar\nadecuadamente al usuario en su búsqueda, adecuando los caminos más apropiados para cada\nusuario de acuerdo con sus objetivos, sus conocimientos previos o sus necesidades y perfiles.\nLa usabilidad en este contexto, se enmarca en una pregunta sobre si una aplicación Web educativa\nes lo suficientemente buena para satisfacer las necesidades y requerimientos del usuario. Es decir,\nla usabilidad correspon

## Separate Problem Sections

In [7]:
problemTSU = [j for j in problemTSU.split("<Problema>")]
problemLic = [j for j in problemLic.split("<Problema>")]
problemMasters = [j for j in problemMasters.split('<Problema>')]
problemPhd = [j for j in problemPhd.split('<Problema>')]
problemTSU
problemLic
problemMasters
problemPhd

['',
 'La gran cantidad de información disponible en Internet, la complejidad y heterogeneidad de la\nmisma, junto con las distintas características de los usuarios que acceden a dicha información, así\ncomo la gran competencia existente en la red, hacen crítico que el desarrollo de aplicaciones\nbasadas en Web, y de manera particular los sitios Web educativos deban contemplar no solo\naspectos de funcionalidad sino también de usabilidad, tomando en cuenta que estas aplicaciones no\nsolo deben tener como objetivo poner a disposición de los usuarios información exacta, sino orientar\nadecuadamente al usuario en su búsqueda, adecuando los caminos más apropiados para cada\nusuario de acuerdo con sus objetivos, sus conocimientos previos o sus necesidades y perfiles.\nLa usabilidad en este contexto, se enmarca en una pregunta sobre si una aplicación Web educativa\nes lo suficientemente buena para satisfacer las necesidades y requerimientos del usuario. Es decir,\nla usabilidad corresponde a

## Structure the Data

In [8]:
TSU_structured = sepMeth(problemTSU)
Lic_structured = sepMeth(problemLic)
Masters_structured = sepMeth(problemMasters)
Phd_structured = sepMeth(problemPhd)
TSU_structured
Lic_structured
Masters_structured
Phd_structured

{'Problema1': {},
 'Problema2': {1: 'La',
  2: 'gran',
  3: 'cantidad',
  4: 'de',
  5: 'información',
  6: 'disponible',
  7: 'en',
  8: 'Internet,',
  9: 'la',
  10: 'complejidad',
  11: 'y',
  12: 'heterogeneidad',
  13: 'de',
  14: 'la',
  15: 'misma,',
  16: 'junto',
  17: 'con',
  18: 'las',
  19: 'distintas',
  20: 'características',
  21: 'de',
  22: 'los',
  23: 'usuarios',
  24: 'que',
  25: 'acceden',
  26: 'a',
  27: 'dicha',
  28: 'información,',
  29: 'así',
  30: 'como',
  31: 'la',
  32: 'gran',
  33: 'competencia',
  34: 'existente',
  35: 'en',
  36: 'la',
  37: 'red,',
  38: 'hacen',
  39: 'crítico',
  40: 'que',
  41: 'el',
  42: 'desarrollo',
  43: 'de',
  44: 'aplicaciones',
  45: 'basadas',
  46: 'en',
  47: 'Web,',
  48: 'y',
  49: 'de',
  50: 'manera',
  51: 'particular',
  52: 'los',
  53: 'sitios',
  54: 'Web',
  55: 'educativos',
  56: 'deban',
  57: 'contemplar',
  58: 'no',
  59: 'solo',
  60: 'aspectos',
  61: 'de',
  62: 'funcionalidad',
  63: 'sino',
  

## Remove Empty Dictionaries

In [9]:
TSU_structured = cleanVector(TSU_structured)
Lic_structured = cleanVector(Lic_structured)
Masters_structured = cleanVector(Masters_structured)
Phd_structured = cleanVector(Phd_structured)
TSU_structured
Lic_structured
Masters_structured
Phd_structured

{'Problema2': {1: 'La',
  2: 'gran',
  3: 'cantidad',
  4: 'de',
  5: 'información',
  6: 'disponible',
  7: 'en',
  8: 'Internet,',
  9: 'la',
  10: 'complejidad',
  11: 'y',
  12: 'heterogeneidad',
  13: 'de',
  14: 'la',
  15: 'misma,',
  16: 'junto',
  17: 'con',
  18: 'las',
  19: 'distintas',
  20: 'características',
  21: 'de',
  22: 'los',
  23: 'usuarios',
  24: 'que',
  25: 'acceden',
  26: 'a',
  27: 'dicha',
  28: 'información,',
  29: 'así',
  30: 'como',
  31: 'la',
  32: 'gran',
  33: 'competencia',
  34: 'existente',
  35: 'en',
  36: 'la',
  37: 'red,',
  38: 'hacen',
  39: 'crítico',
  40: 'que',
  41: 'el',
  42: 'desarrollo',
  43: 'de',
  44: 'aplicaciones',
  45: 'basadas',
  46: 'en',
  47: 'Web,',
  48: 'y',
  49: 'de',
  50: 'manera',
  51: 'particular',
  52: 'los',
  53: 'sitios',
  54: 'Web',
  55: 'educativos',
  56: 'deban',
  57: 'contemplar',
  58: 'no',
  59: 'solo',
  60: 'aspectos',
  61: 'de',
  62: 'funcionalidad',
  63: 'sino',
  64: 'también',
  6

## Obtain Size for the Justification Data

In [10]:
sizesTSU = returnSizes(TSU_structured)
sizesLic = returnSizes(Lic_structured)
sizesMasters = returnSizes(Masters_structured)
sizesPhd = returnSizes(Phd_structured)
sizesTSU
sizesLic 
sizesMasters
sizesPhd

{'Smallest': 43, 'Biggest': 1642, 'Average': 588.3636363636364}

## Remove Stop Words and Cleaning

In [11]:
TSU_structured = cleanMeth(sw, TSU_structured)
Lic_structured = cleanMeth(sw, Lic_structured)
Masters_structured = cleanMeth(sw, Masters_structured)
Phd_structured = cleanMeth(sw, Phd_structured)
TSU_structured
Lic_structured
Masters_structured
Phd_structured

{'Problema2': {2: 'gran',
  3: 'cantidad',
  5: 'información',
  6: 'disponible',
  8: 'internet',
  10: 'complejidad',
  12: 'heterogeneidad',
  15: 'misma',
  16: 'junto',
  19: 'distintas',
  20: 'características',
  23: 'usuarios',
  25: 'acceden',
  27: 'dicha',
  28: 'información',
  29: 'así',
  32: 'gran',
  33: 'competencia',
  34: 'existente',
  37: 'red',
  38: 'hacen',
  39: 'crítico',
  42: 'desarrollo',
  44: 'aplicaciones',
  45: 'basadas',
  47: 'web',
  50: 'manera',
  51: 'particular',
  53: 'sitios',
  54: 'web',
  55: 'educativos',
  56: 'deban',
  57: 'contemplar',
  59: 'solo',
  60: 'aspectos',
  62: 'funcionalidad',
  63: 'sino',
  66: 'usabilidad',
  67: 'tomando',
  69: 'cuenta',
  72: 'aplicaciones',
  74: 'solo',
  75: 'deben',
  76: 'tener',
  78: 'objetivo',
  79: 'poner',
  81: 'disposición',
  84: 'usuarios',
  85: 'información',
  86: 'exacta',
  87: 'sino',
  88: 'orientar',
  89: 'adecuadamente',
  91: 'usuario',
  94: 'búsqueda',
  95: 'adecuando',
 

## Create a Dictionary: Justification#-List of Words (Easier to handle)

In [12]:
TSU_structured = listMeth(TSU_structured)
Lic_structured = listMeth(Lic_structured)
Masters_structured = listMeth(Masters_structured)
Phd_structured = listMeth(Phd_structured)

TSU_structured
Lic_structured
Masters_structured
Phd_structured

{'Problema2': ['gran',
  'cantidad',
  'información',
  'disponible',
  'internet',
  'complejidad',
  'heterogeneidad',
  'misma',
  'junto',
  'distintas',
  'características',
  'usuarios',
  'acceden',
  'dicha',
  'información',
  'así',
  'gran',
  'competencia',
  'existente',
  'red',
  'hacen',
  'crítico',
  'desarrollo',
  'aplicaciones',
  'basadas',
  'web',
  'manera',
  'particular',
  'sitios',
  'web',
  'educativos',
  'deban',
  'contemplar',
  'solo',
  'aspectos',
  'funcionalidad',
  'sino',
  'usabilidad',
  'tomando',
  'cuenta',
  'aplicaciones',
  'solo',
  'deben',
  'tener',
  'objetivo',
  'poner',
  'disposición',
  'usuarios',
  'información',
  'exacta',
  'sino',
  'orientar',
  'adecuadamente',
  'usuario',
  'búsqueda',
  'adecuando',
  'caminos',
  'apropiados',
  'cada',
  'usuario',
  'acuerdo',
  'objetivos',
  'conocimientos',
  'previos',
  'necesidades',
  'perfiles',
  'usabilidad',
  'contexto',
  'enmarca',
  'pregunta',
  'si',
  'aplicació

## Select a specific number of vectors to work with

In [13]:
TSU_structured = getNVector(len(TSU_structured), TSU_structured)
Lic_structured = getNVector(len(Lic_structured), Lic_structured)
Masters_structured = getNVector(len(Masters_structured), Masters_structured)
Phd_structured = getNVector(len(Phd_structured), Phd_structured)

len(TSU_structured)
len(Lic_structured)
len(Masters_structured)
len(Phd_structured)

33

## Divide our data in eighty and twenty percent (This in order to have some experimental data)

In [14]:
twenty_TSU, eighty_TSU = divideEightyTwenty(TSU_structured)
twenty_Lic, eighty_Lic = divideEightyTwenty(Lic_structured)
twenty_Masters, eighty_Masters = divideEightyTwenty(Masters_structured)
twenty_Phd, eighty_Phd = divideEightyTwenty(Phd_structured)

len(twenty_TSU)
len(eighty_TSU)
len(twenty_Lic)
len(eighty_Lic)
len(twenty_Masters)
len(eighty_Masters)
len(twenty_Phd)
len(eighty_Phd)

26

## Obtain the Relative Frequency

In [15]:
twentyRFTSU = relFreq(twenty_TSU)
eightyRFTSU = relFreq(eighty_TSU)
twentyRFLic = relFreq(twenty_Lic)
eightyRFLic = relFreq(eighty_Lic)
twentyRFMasters = relFreq(twenty_Masters)
eightyRFMasters = relFreq(eighty_Masters)
twentyRFPhd = relFreq(twenty_Phd)
eightyRFPhd = relFreq(eighty_Phd)

len(twentyRFTSU)
len(eightyRFTSU)
len(twentyRFLic)
len(eightyRFLic)
len(twentyRFMasters)
len(eightyRFMasters)
len(twentyRFPhd)
len(eightyRFPhd)

26

## Most Common Spanish Words

In [16]:
mostCommonDic = mostCommon()
mostCommonDic

{'Frec.absoluta': 'Frec.normalizada',
 'de': '9999518',
 'la': '6277560',
 'que': '4681839',
 'el': '4569652',
 'en': '4234281',
 'y': '4180279',
 'a': '3260939',
 'los': '2618657',
 'se': '2022514',
 'del': '1857225',
 'las': '1686741',
 'un': '1659827',
 'por': '1561904',
 'con': '1481607',
 'no': '1465503',
 'una': '1347603',
 'su': '1103617',
 'para': '1062152',
 'es': '1019669',
 'al': '951054',
 'lo': '866955',
 'como': '773465',
 'más': '661696',
 'o': '542284',
 'pero': '450512',
 'sus': '449870',
 'le': '413241',
 'ha': '380339',
 'me': '374368',
 'si': '327480',
 'sin': '298383',
 'sobre': '289704',
 'este': '285461',
 'ya': '274177',
 'entre': '267493',
 'cuando': '257272',
 'todo': '247340',
 'esta': '238841',
 'ser': '232924',
 'son': '232415',
 'dos': '228439',
 'también': '227411',
 'fue': '223791',
 'había': '223430',
 'era': '219933',
 'muy': '208540',
 'años': '203027',
 'hasta': '202935',
 'desde': '198647',
 'está': '194168',
 'mi': '186360',
 'porque': '185700',
 '

## Obtain the Log of the Common Value

In [17]:
twentyLFTSU = logFreq(twenty_TSU, mostCommonDic)
eightyLFTSU = logFreq(eighty_TSU, mostCommonDic)
twentyLFLic = logFreq(twenty_Lic, mostCommonDic)
eightyLFLic = logFreq(eighty_Lic, mostCommonDic)
twentyLFMasters = logFreq(twenty_Masters, mostCommonDic)
eightyLFMasters = logFreq(eighty_Masters, mostCommonDic)
twentyLFPhd = logFreq(twenty_Phd, mostCommonDic)
eightyLFPhd = logFreq(eighty_Phd, mostCommonDic)

twentyLFTSU
eightyLFTSU
twentyLFLic
eightyLFLic
twentyLFMasters
eightyLFMasters
twentyLFPhd
eightyLFPhd

{'Problema9': {'minería': 0.0,
  'texto': 9.782110566369528,
  'área': 9.640107772607312,
  'investigación': 10.165890277802454,
  'reciente': 0.0,
  'procesamiento': 0.0,
  'textos': 0.0,
  'define': 0.0,
  'proceso': 10.780288735123806,
  'descubrimiento': 0.0,
  'patrones': 0.0,
  'interesantes': 0.0,
  'colección': 0.0,
  'deben': 10.240245462634501,
  'existir': 0.0,
  'explícitamente': 0.0,
  'ningún': 10.34051592510041,
  'surgir': 0.0,
  'relacionar': 0.0,
  'contenido': 9.647368744540465,
  'varios': 10.472063081843023,
  'hearst': 0.0,
  'kodratoff': 0.0,
  'consiste': 0.0,
  'dos': 12.339024495252145,
  'etapas': 0.0,
  'principales': 9.82498532438117,
  'etapa': 9.596690616224665,
  'preprocesamiento': 0.0,
  'tan': 11.630450689618431,
  'primera': 11.331739263416525,
  'transforman': 0.0,
  'algún': 10.47908916924103,
  'tipo': 10.94387026564006,
  'representación': 0.0,
  'estructurada': 0.0,
  'semiestructurada': 0.0,
  'facilite': 0.0,
  'posterior': 0.0,
  'análisis': 

## Create the Vocabulary

In [18]:
voc = obtainVocSingle(TSU_structured)
voc = obtainVocDouble(TSU_structured, Lic_structured)
voc = obtainVoc(TSU_structured, Lic_structured, Masters_structured, Phd_structured)
len(voc)

11733

## Padding (Dimensionate the Vectors based on the size of the Vocabulary)

In [19]:
# For TSU
twentyPadRFTSU = dimRelFreq(twentyRFTSU, voc)
twentyPadLFTSU = dimRelFreq(twentyLFTSU, voc)
eightyPadRFTSU = dimRelFreq(eightyRFTSU, voc)
eightyPadLFTSU = dimRelFreq(eightyLFTSU, voc)
twentyPadRFTSU
eightyPadRFTSU
twentyPadLFTSU
eightyPadLFTSU
# For Lic
twentyPadRFLic = dimRelFreq(twentyRFLic, voc)
twentyPadLFLic = dimRelFreq(twentyLFLic, voc)
eightyPadRFLic = dimRelFreq(eightyRFLic, voc)
eightyPadLFLic = dimRelFreq(eightyLFLic, voc)
twentyPadRFLic
twentyPadLFLic
eightyPadRFLic
eightyPadLFLic
# For Masters
twentyPadRFMasters = dimRelFreq(twentyRFMasters, voc)
twentyPadLFMasters = dimRelFreq(twentyLFMasters, voc)
eightyPadRFMasters = dimRelFreq(eightyRFMasters, voc)
eightyPadLFMasters = dimRelFreq(eightyLFMasters, voc)
# For Phd
twentyPadRFPhd = dimRelFreq(twentyRFPhd, voc)
twentyPadLFPhd = dimRelFreq(twentyLFPhd, voc)
eightyPadRFPhd = dimRelFreq(eightyRFPhd, voc)
eightyPadLFPhd = dimRelFreq(eightyLFPhd, voc)

## Concatenate Local and Global Vectors

In [20]:
# TSU
twentyConTSU = concatenateDictionaries(twentyPadRFTSU, twentyPadLFTSU)
eightyConTSU = concatenateDictionaries(eightyPadRFTSU, eightyPadLFTSU)
twentyConLic = concatenateDictionaries(twentyPadRFLic, twentyPadLFLic)
eightyConLic = concatenateDictionaries(eightyPadRFLic, eightyPadLFLic)
twentyConMasters = concatenateDictionaries(twentyPadRFMasters, twentyPadLFMasters)
eightyConMasters = concatenateDictionaries(eightyPadRFMasters, eightyPadLFMasters)
twentyConPhd = concatenateDictionaries(twentyPadRFPhd, twentyPadLFPhd)
eightyConPhd = concatenateDictionaries(eightyPadRFPhd, eightyPadLFPhd)
np.array((twentyConLic)).sum()
np.array((eightyConLic)).sum()
np.array((twentyConMasters)).sum()
np.array((eightyConMasters)).sum()
np.array((twentyConPhd)).sum()
np.array((eightyConPhd)).sum()

17245.27181305777

## Expand and Classify Data

In [21]:
# HST and Undergraduate
twentyPOneTL, twentyMOneTL = comConData(twentyConTSU, twentyConLic)
eightyPOneTL, eightyMOneTL = comConData(eightyConTSU, eightyConLic)
np.array((twentyPOneTL)).sum()
np.array((twentyMOneTL)).sum()
# HST and Masters
twentyPOneTM, twentyMOneTM = comConData(twentyConTSU, twentyConMasters)
eightyPOneTM, eightyMOneTM = comConData(eightyConTSU, eightyConMasters)
# HST and Phd
twentyPOneTP, twentyMOneTP = comConData(twentyConTSU, twentyConPhd)
eightyPOneTP, eightyMOneTP = comConData(eightyConTSU, eightyConPhd)
# Undergraduate and Masters
twentyPOneUM, twentyMOneUM = comConData(twentyConLic, twentyConMasters)
eightyPOneUM, eightyMOneUM = comConData(eightyConLic, eightyConMasters)
# Undergraduate and Phd
twentyPOneUP, twentyMOneUP = comConData(twentyConLic, twentyConPhd)
eightyPOneUP, eightyMOneUP = comConData(eightyConLic, eightyConPhd)
# Masters and Phd
twentyPOneMP, twentyMOneMP = comConData(twentyConMasters, twentyConPhd)
eightyPOneMP, eightyMOneMP = comConData(eightyConMasters, eightyConPhd)

## Append the Data and Create Classification Vector

In [22]:
# HST and Undergraduate
TLM_test, tly_test = allTogetherNowDouble(twentyPOneTL, twentyMOneTL)
len(TLM_test)
len(tly_test)
TLM_train, tly_train = allTogetherNowDouble(eightyPOneTL, eightyMOneTL)
len(TLM_train)
len(tly_train)
# HST and Masters
TMM_test, tmy_test = allTogetherNowDouble(twentyPOneTM, twentyMOneTM)
TMM_train, tmy_train = allTogetherNowDouble(eightyPOneTM, eightyMOneTM)
#HST and Phd
TPM_test, tpy_test = allTogetherNowDouble(twentyPOneTP, twentyMOneTP)
TPM_train, tpy_train = allTogetherNowDouble(eightyPOneTP, eightyMOneTP)
#Undergraduate and Masters
UMM_test, umy_test = allTogetherNowDouble(twentyPOneUM, twentyMOneUM)
UMM_train, umy_train = allTogetherNowDouble(eightyPOneUM, eightyMOneUM)
#Undergraduate and PhD
UPM_test, upy_test = allTogetherNowDouble(twentyPOneUP, twentyMOneUP)
UPM_train, upy_train = allTogetherNowDouble(eightyPOneUP, eightyMOneUP)
#Masters and PhD
MPM_test, mpy_test = allTogetherNowDouble(twentyPOneMP, twentyMOneMP)
MPM_train, mpy_train = allTogetherNowDouble(eightyPOneMP, eightyMOneMP)

In [23]:
len(TMM_test[0])
len(TMM_train[0])

46932

# HST and Udergraduate

## SVM Classifier

In [103]:
svclassifier = svm.SVC()
svclassifier.fit(TLM_train, tly_train)

/Users/josemedardotapiatellez/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [104]:
y_pred = svclassifier.predict(TLM_test)

print("The accuracy value for the SVM classifier is: {} %".format(obtain_accuracy(tly_test, y_pred)*100))

The accuracy value for the SVM classifier is: 35.08403361344538 %


## K Nearest Neighbors (KNN)

In [105]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors = 3)
neigh.fit(TLM_train, tly_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [106]:
y_pred = neigh.predict(TLM_test)
print('The accurracy value for the KNN classifier is: {} %'.format(obtain_accuracy(tly_test, y_pred)*100))

The accurracy value for the KNN classifier is: 44.537815126050425 %


## Multilayer Perceptron

In [107]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha = 1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(TLM_train, tly_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5, 2), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [108]:
y_pred = clf.predict(TLM_test)
print("The accuracy value for the Multilayer Perceptron is: {} %".format(obtain_accuracy(tly_test, y_pred)*100))

The accuracy value for the Multilayer Perceptron is: 41.38655462184874 %


# HST and Masters

## SVM Classifier

In [48]:
svclassifier = svm.SVC()
svclassifier.fit(TMM_train, tmy_train)

/Users/josemedardotapiatellez/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [49]:
y_pred = svclassifier.predict(TMM_test)
print("The accuracy value for the SVM classifier is: {} %".format(obtain_accuracy(tmy_test, y_pred)*100))

The accuracy value for the SVM classifier is: 52.77777777777778 %


## K Nearest Neighbors (KNN)

In [50]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors = 3)
neigh.fit(TMM_train, tmy_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [51]:
y_pred = neigh.predict(TMM_test)
print('The accurracy value for the KNN classifier is: {} %'.format(obtain_accuracy(tmy_test, y_pred)*100))

The accurracy value for the KNN classifier is: 56.290849673202615 %


## Multilayer Perceptron

In [52]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha = 1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(TMM_train, tmy_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5, 2), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [53]:
y_pred = clf.predict(TMM_test)
print("The accuracy value for the Multilayer Perceptron is: {} %".format(obtain_accuracy(tmy_test, y_pred)*100))

The accuracy value for the Multilayer Perceptron is: 61.356209150326805 %


# HST and PhD

## SVM Classifier

In [24]:
svclassifier = svm.SVC()
svclassifier.fit(TPM_train, tpy_train)

/Users/josemedardotapiatellez/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [25]:
y_pred = svclassifier.predict(TPM_test)
print("The accuracy value for the SVM classifier is: {} %".format(obtain_accuracy(tpy_test, y_pred)*100))

The accuracy value for the SVM classifier is: 79.83193277310924 %


## K Nearest Neighbors (KNN)

In [26]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors = 3)
neigh.fit(TPM_train, tpy_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [27]:
y_pred = neigh.predict(TPM_test)
print('The accurracy value for the KNN classifier is: {} %'.format(obtain_accuracy(tpy_test, y_pred)*100))

The accurracy value for the KNN classifier is: 57.14285714285714 %


## Multilayer Perceptron

In [28]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha = 1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(TPM_train, tpy_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5, 2), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [29]:
y_pred = clf.predict(TPM_test)
print("The accuracy value for the Multilayer Perceptron is: {} %".format(obtain_accuracy(tpy_test, y_pred)*100))

The accuracy value for the Multilayer Perceptron is: 86.97478991596638 %


# Undergraduate and Masters

## SVM Classifier

In [30]:
svclassifier = svm.SVC()
svclassifier.fit(UMM_train, umy_train)

/Users/josemedardotapiatellez/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [31]:
y_pred = svclassifier.predict(UMM_test)
print("The accuracy value for the SVM classifier is: {} %".format(obtain_accuracy(umy_test, y_pred)*100))

The accuracy value for the SVM classifier is: 57.341269841269835 %


## K Nearest Neighbors (KNN)

In [32]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors = 3)
neigh.fit(UMM_train, umy_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [33]:
y_pred = neigh.predict(UMM_test)
print('The accurracy value for the KNN classifier is: {} %'.format(obtain_accuracy(umy_test, y_pred)*100))

The accurracy value for the KNN classifier is: 52.083333333333336 %


## Multilayer Perceptron

In [34]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha = 1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(UMM_train, umy_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5, 2), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [35]:
y_pred = clf.predict(UMM_test)
print("The accuracy value for the Multilayer Perceptron is: {} %".format(obtain_accuracy(umy_test, y_pred)*100))

The accuracy value for the Multilayer Perceptron is: 68.45238095238095 %


# Undergraduate and PhD

## SVM Classifier

In [36]:
svclassifier = svm.SVC()
svclassifier.fit(UPM_train, upy_train)

/Users/josemedardotapiatellez/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [37]:
y_pred = svclassifier.predict(UPM_test)
print("The accuracy value for the SVM classifier is: {} %".format(obtain_accuracy(upy_test, y_pred)*100))

The accuracy value for the SVM classifier is: 74.48979591836735 %


## K Nearest Neighbors (KNN)

In [38]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors = 3)
neigh.fit(UPM_train, upy_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [39]:
y_pred = neigh.predict(UPM_test)
print('The accurracy value for the KNN classifier is: {} %'.format(obtain_accuracy(upy_test, y_pred)*100))

The accurracy value for the KNN classifier is: 40.816326530612244 %


## Multilayer Perceptron

In [40]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha = 1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(UPM_train, upy_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5, 2), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [41]:
y_pred = clf.predict(UPM_test)
print("The accuracy value for the Multilayer Perceptron is: {} %".format(obtain_accuracy(upy_test, y_pred)*100))

The accuracy value for the Multilayer Perceptron is: 64.79591836734694 %


# Masters and PhD

## SVM Classifier

In [42]:
svclassifier = svm.SVC()
svclassifier.fit(MPM_train, mpy_train)

/Users/josemedardotapiatellez/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [43]:
y_pred = svclassifier.predict(MPM_test)
print("The accuracy value for the SVM classifier is: {} %".format(obtain_accuracy(mpy_test, y_pred)*100))

The accuracy value for the SVM classifier is: 69.04761904761905 %


## K Nearest Neighbors (KNN)

In [44]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors = 3)
neigh.fit(MPM_train, mpy_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [45]:
y_pred = neigh.predict(MPM_test)
print('The accurracy value for the KNN classifier is: {} %'.format(obtain_accuracy(mpy_test, y_pred)*100))

The accurracy value for the KNN classifier is: 50.79365079365079 %


## Multilayer Perceptron

In [46]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha = 1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(MPM_train, mpy_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5, 2), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [47]:
y_pred = clf.predict(MPM_test)
print("The accuracy value for the Multilayer Perceptron is: {} %".format(obtain_accuracy(mpy_test, y_pred)*100))

The accuracy value for the Multilayer Perceptron is: 79.16666666666666 %
